**Ansambli** (ensemble) je grupa modela u masinskom ucenju koji predstavljaju skupove veceg broja modela koji zajednicki donose odluke. Razlog njihove upotrebe pociva na tome da veci broj pravilno konstruisanih modela moze dati znacajno bolju preciznost od samo jednog modela.

Dve familije modela:
* prosta agregacija (bagging)
* pojacavanje (boosting)

**Ideja:** nezavisne greske se ponistavaju! [Centralna granicna terema](https://en.wikipedia.org/wiki/Central_limit_theorem) garantuje smanjenu varijansu proseka. <br>
**Prosta agregacija** podrazumeva treniranje veceg broja modela cije su greske nezavisne. Prilikom predvidanja, svi modeli nude svoja predvidanja, koja se agregiraju kako bi se dobilo predvidanje ansambla.<br> Ukoliko se radi o regresiji, agregacija je najcesce uprosecavanje, a u slucaju klasifikacije glasanje (voting). <br>
**Kompromis:** broj modela sa jedne strane i preprilagodjavanje i komleksnost sa druge.

[Bootstrapping](https://en.wikipedia.org/wiki/Bootstrapping_(statistics))

<img src='bootstrap.png'>

<img src='bagging.jpeg' width="500"/>

Jedan od najpoznatijih modela ove vrste jeste:

### Random forest 

<img src='random_forest.png'>

Ansambl, odnosno slucajna suma, se sastoji od m stabala treniranih na razlicitim podskupovima skupa za treniranje. Jedno stablo se trenira tako sto se izabere podskup skupa za treniranje zadate velicine (**Bootstrap Aggregation**). Bootstrap uzorak se bira od svih instanci, ali sa ponavljanjem. Tako da sve instance ucestvuju, ali se uzorci razlikuju. Broj stabala m se moze posmatrati i kao regularizacioni parametar. Takodje, moze se uzeti i samo podskup skupa atributa u svakom cvoru pre testa podele (**Feature Randomness**). Ova dva metoda prilikom treniranja doprinose tome da imamo sto manje korelisana stabla , odnosno njihove greske, sto je i cilj agregacije. <br>
Treniranje generalno nije kompleksno, a preciznost predvidanja je obicno medu najboljim za ovakve slucajeve upotrebe. Slucajna suma, kao i drugi ansambli, nije interpretabilna.

Funkcije za rad sa anamblima se nalaze u `ensamble` paketu scikit-learn biblioteke.

In [ ]:
from sklearn import ensemble

Parametri:
* `n_estimators parametar` - broj stabala koja se treniraju
* svojstva stabala iz prethodne lekcije (kriterijum homogenosti, maksimalna dubina stabla, maksimalni broj atributa,..)
* `max_samples` - velicina podskupa instanci nad kojim se stabla treniraju
* `random_state` - zbog reprodukicje jer imamo slucajan izbor instanci, kao i atributa prilikom podele

In [ ]:
model_forest = ensemble.RandomForestClassifier(n_estimators=20, max_depth=3, random_state=42) # izbor parametara se vrsi validacijom (uskoro!)

In [ ]:
%run DecisionTrees.ipynb

In [ ]:
model_forest.fit(X_train, y_train)

In [ ]:
y_pred = model_forest.predict(X_test)

### Evaluacija 

In [ ]:
metrics.accuracy_score(y_test, y_pred)

In [ ]:
metrics.confusion_matrix(y_test, y_pred)

In [ ]:
print(metrics.classification_report(y_test, y_pred))

In [ ]:
y_pred_proba = model_forest.predict_proba(X_test)[::,1]

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test,  y_pred_proba) 

In [ ]:
auc = metrics.roc_auc_score(y_test, y_pred_proba) 

In [ ]:
auc

In [ ]:
plt.plot(fpr, tpr, label="AUC="+str(auc))
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.legend(loc=4)
plt.show()

### Znacajnost atributa

Znacajnost atributa odrejujemo kao prosek njihovih znacajnosti u svim stablima u sumi.

In [ ]:
plt.barh(list(X.columns), model_forest.feature_importances_)
plt.show()

### Drugi nacin

Random forest je samo posebna klasa modela proste agregacije, pa cemo ga sada implementirati na taj nacin. Isto ovako, mozemo napraviti i ansambl nekih drugih modela (istog tipa) koji nisu stabla odlucivanja.

Za kreiranje proste agregacije koristi se `BaggingClassifier` metoda `ensemble` paketa. Ona od parametara, pored gorenavedenih, ocekuje i osnovni model koji se koristi u ansamblu. Takodje, parametrima `booststrap` i `bootstrap_features` se kontrolise uzorkovanje sa ili bez ponavljanja. 

In [ ]:
model_bagging = ensemble.BaggingClassifier(tree.DecisionTreeClassifier(random_state=42),
                                        n_estimators=500, max_samples=100, bootstrap=True, random_state=42)

In [ ]:
model_bagging.fit(X_train, y_train)

In [ ]:
y_pred = model_bagging.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, y_pred))